# 암종 분류 모델 개발: 변수 선택 및 기준 모델 설정

## 1. 변수 인코딩 전략

### 범주형 변수가 아닌 경우의 인코딩 방법

- **수치형 변수**: 그대로 사용 (필요시 정규화/표준화)
- **순서형 변수**: 순서를 반영한 정수 인코딩
- **텍스트 데이터**: TF-IDF 또는 워드 임베딩

### 유전자 변이 데이터 처리 방법

| 방법 | 장점 | 단점 |
|------|------|------|
| 원핫인코딩 | 정보 손실 없음 | 차원이 매우 커질 수 있음 |
| 희소행렬 | 메모리 효율적 | 원핫인코딩과 유사한 특성 |
| 임베딩 | 의미적 관계 캡처 가능 | 충분한 훈련 데이터 필요 |

### 제안 단계

1. Label Encoding으로 기준 모델 생성
2. 원핫인코딩 적용 후 성능 비교
3. 대규모 데이터의 경우 희소행렬 고려
4. 충분한 데이터 확보 시 임베딩 방법 시도

## 2. 기준 모델 선택: RandomForest vs LGBM

### RandomForest

#### 장점
- 과적합에 강하고 안정적
- 하이퍼파라미터 튜닝의 중요도 낮음
- 특성 중요도 쉽게 획득 가능

#### 단점
- 대규모 데이터셋에서 속도 저하
- 높은 메모리 사용량

### LGBM

#### 장점
- 대규모 데이터셋에서 빠른 학습/예측
- 메모리 효율적
- 일반적으로 높은 성능

#### 단점
- 과적합 위험 높음 (세심한 튜닝 필요)
- 소규모 데이터셋에서 불안정할 수 있음

### 기준 모델로 RandomForest 선택 이유

1. 높은 안정성으로 초기 모델에 적합
2. 간단한 하이퍼파라미터로 빠른 기준 성능 획득
3. 용이한 특성 중요도 획득으로 초기 특성 선택에 유용

## 권장 접근 방식

1. RandomForest로 기준 성능 설정
2. LGBM을 활용한 성능 개선 시도
   - 주의: 적절한 하이퍼파라미터 튜닝 필요


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [6]:
# 1. 데이터 로드
file_path = '/content/drive/MyDrive/생명연구자원  AI 활용 경진대회/Dataset/train.csv'
data = pd.read_csv(file_path)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6201 entries, 0 to 6200
Columns: 4386 entries, ID to ZYX
dtypes: int64(1), object(4385)
memory usage: 207.5+ MB


In [10]:
data.columns

Index(['ID', 'SUBCLASS', 'A2M', 'AAAS', 'AADAT', 'AARS1', 'ABAT', 'ABCA1',
       'ABCA2', 'ABCA3',
       ...
       'ZNF292', 'ZNF365', 'ZNF639', 'ZNF707', 'ZNFX1', 'ZNRF4', 'ZPBP',
       'ZW10', 'ZWINT', 'ZYX'],
      dtype='object', length=4386)

In [11]:
# 2. 레이블 인코딩
le = LabelEncoder()
data['SUBCLASS'] = le.fit_transform(data['SUBCLASS'])

In [12]:
# 3. 변이 정보 인코딩
def encode_mutation(x):
    if x == 'WT':
        return 0
    else:
        return 1

features = data.drop(['ID', 'SUBCLASS'], axis=1)
features = features.applymap(encode_mutation)

<ipython-input-12-d341f831b775>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features = features.applymap(encode_mutation)


In [13]:
# 4. 입력 변수와 타겟 변수 분리
X = features.values
y = data['SUBCLASS'].values

# 5. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# 5. 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [16]:
# 6. 모델 평가
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.28444802578565676
F1 Score: 0.2510428371714494

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.57      0.67        14
           1       0.00      0.00      0.00        27
           2       0.25      0.46      0.32       149
           3       0.50      0.03      0.06        31
           4       0.61      0.63      0.62        30
           5       1.00      0.10      0.18        10
           6       0.23      0.26      0.25       116
           7       0.26      0.12      0.16        51
           8       0.19      0.20      0.19        86
           9       0.02      0.02      0.02        54
          10       0.34      0.64      0.44        39
          11       0.12      0.14      0.13        49
          12       0.75      0.09      0.16        34
          13       0.60      0.08      0.14        37
          14       0.33      0.03      0.05        39
          15       0.31      0.31      0.31        5

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
# 7. 특성 중요도 추출
importances = model.feature_importances_
gene_list = features.columns
feature_importance = pd.Series(importances, index=gene_list)
feature_importance = feature_importance.sort_values(ascending=False)

print('Top 10 Important Genes:')
print(feature_importance.head(10))

Top 10 Important Genes:
BRAF      0.024143
IDH1      0.018800
VHL       0.017077
PIK3CA    0.013305
TP53      0.011870
APC       0.010249
ATRX      0.005468
NPM1      0.005124
PTEN      0.005061
GATA3     0.004324
dtype: float64


In [40]:
feature_importance[feature_importance > 0.0002]

,0
BRAF,0.024143
IDH1,0.018800
VHL,0.017077
PIK3CA,0.013305
TP53,0.011870
...,...
MOV10,0.000200
STK39,0.000200
FBP2,0.000200
CYP2S1,0.000200


In [42]:
feature_importance.iloc[:1543]

,0
BRAF,0.024143
IDH1,0.018800
VHL,0.017077
PIK3CA,0.013305
TP53,0.011870
...,...
MOV10,0.000200
STK39,0.000200
FBP2,0.000200
CYP2S1,0.000200


In [44]:
# 9. 선택된 특성으로 모델 재학습
N = 1543  # 선택할 특성의 수
selected_features = data[feature_importance.index[:N]]
selected_features = features.applymap(encode_mutation)
X_selected = selected_features.values
X_train_selected, X_test_selected = train_test_split(X_selected, test_size=0.2, random_state=42)

<ipython-input-44-e634103ef552>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  selected_features = features.applymap(encode_mutation)


In [45]:
model_selected = RandomForestClassifier(n_estimators=100, random_state=42)
model_selected.fit(X_train_selected, y_train)

RandomForestClassifier(random_state=42)

In [46]:
# 10. 선택된 특성으로 모델 평가
y_pred_selected = model_selected.predict(X_test_selected)
print("\nAccuracy (with selected features):", accuracy_score(y_test, y_pred_selected))
print("F1 Score (with selected features):", f1_score(y_test, y_pred_selected, average='macro'))
print("\nClassification Report (with selected features):")
print(classification_report(y_test, y_pred_selected))


Accuracy (with selected features): 0.1200644641418211
F1 Score (with selected features): 0.008245711123408965

Classification Report (with selected features):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        27
           2       0.12      1.00      0.21       149
           3       0.00      0.00      0.00        31
           4       0.00      0.00      0.00        30
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00       116
           7       0.00      0.00      0.00        51
           8       0.00      0.00      0.00        86
           9       0.00      0.00      0.00        54
          10       0.00      0.00      0.00        39
          11       0.00      0.00      0.00        49
          12       0.00      0.00      0.00        34
          13       0.00      0.00      0.00        37
          14       0.00      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
# 11. 특성 중요도 저장
file_path = '/content/drive/MyDrive/생명연구자원  AI 활용 경진대회/Dataset/'
feature_importance.to_csv(file_path + 'RF_feature_importance.csv')